In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
cap = cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip=0
face_data=[]

dataset_path='./data/'

file_name = input("Enter the name of person:")
while True:
    
    ret,frame = cap.read()
    if ret==False:
        continue
    fray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #list of faces is "faces"
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    faces=sorted(faces,key=lambda f:f[2]*f[3])
    
    for face in faces[-1:]:
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

        #Cropping the required part
        offset=10
        face_section= frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        skip+=1
        
        if(skip%10==0):
            face_data.append(face_section)
            print(len(face_data))
    
    cv2.imshow("Frame",frame)
    cv2.imshow("Face section" , face_section)

    if(skip%10==0):
        pass 
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if(key_pressed == ord('q')):
        break
        
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0]),-1)
print(face_data.shape)
        
np.save(dataset_path+file_name+'.npy',face_data)
print("Data Saved : " ,dataset_path+file_name+'.npy')
cap.release()
cv2.destroyAllWindows()


Enter the name of person:deepanshu
1
2
3
4
5
6
7
8
9
(9, 30000)
Data Saved :  ./data/deepanshu.npy
